In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.plotly as py
import plotly.tools as tls

In [ ]:
data = pd.read_csv("//Users/carolinetroude/Desktop/finaldataset.csv",sep=",",encoding="utf-8",engine="python")

In [ ]:
data.head()

In [ ]:
from stop_words import get_stop_words

stop_en = get_stop_words('en')
print(stop_en)

In [ ]:
data["VALID"] = data["COMMENT"].isnull()
data=data[~data["VALID"]]

In [ ]:
set(list(data["COMMENT"].apply(lambda x: type(x))))

In [ ]:
def clean_text(x):
    words = x.lower().split()
    words = [mot for mot in words if not mot in stop_en]
    words = [mot for mot in words if len(mot)>1]
    return words

data["CLEANED_TEXT"] = data["COMMENT"].apply(clean_text)

In [ ]:
len(data)

In [ ]:
len(data["CLEANED_TEXT"])

In [ ]:
len(data.groupby(["ADDRESS"]).mean())

In [ ]:
data.groupby(["GRADE"])["CLEANED_TEXT"].head()

In [ ]:
data.groupby(["GRADE_COMMENT"])["COMMENT"].apply(lambda x: "".join(x)).reset_index()

In [ ]:
commentaires_par_grade = data.groupby(["GRADE_COMMENT"])["COMMENT"].apply(lambda x: "".join(x)).reset_index()
commentaires_par_grade["CLEANED_TEXT"] = commentaires_par_grade["COMMENT"].apply(clean_text)
commentaires_par_grade2 = data.groupby(["GRADE_COMMENT"])["CLEANED_TEXT"].apply(lambda x: "".join(str(x))).reset_index()

In [ ]:
type(commentaires_par_grade["COMMENT"][0])

In [ ]:
commentaires_par_resto = data.groupby(["ADDRESS"])["COMMENT"].apply(lambda x: "".join(x)).reset_index()
commentaires_par_resto["CLEANED_TEXT"] = commentaires_par_resto["COMMENT"].apply(clean_text)

In [ ]:
len(commentaires_par_resto)

In [ ]:
sentences = data["CLEANED_TEXT"].values

In [ ]:
len(sentences)

In [ ]:
import gensim

model = gensim.models.Word2Vec(sentences, size=200, window=5, min_count=10)

In [ ]:
dict_id_to_word = dict(zip(range(len(model.index2word)),model.Word2Vec.index2word))
dict_word_to_id = dict(zip(model.index2word,range(len(model.wv.index2word))))

In [ ]:
model.wv.syn0.shape

In [ ]:
from bhtsne import *
tsne_proj = tsne(np.float64(model.wv.syn0),perplexity=30)

In [ ]:
proj_df = pd.DataFrame(data={"X_TSNE":tsne_proj[:,0], "Y_TSNE":tsne_proj[:,1]})

In [ ]:
from sklearn.cluster import KMeans

N_clusters=35
kmeans_model = KMeans(n_clusters=N_clusters, random_state=np.random.randint(256),n_jobs=20,verbose=0).fit(model.wv.syn0)
classes = kmeans_model.labels_
proj_df["CLASS"] = classes

In [ ]:
proj_df

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
%matplotlib inline

colormap = get_cmap('plasma')

plt.style.use('ggplot')

plt.figure(figsize=(5,5))
plt.scatter(30,30,s=10,c=[colormap(x) for x in proj_df["CLASS"]])
plt.axis([-30,30,-30,30])
plt.show()

In [ ]:
proj_df.tail()

In [ ]:
len(proj_df)

In [ ]:
sentences

In [ ]:
import gensim

dictionary = gensim.corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=35)

In [ ]:
for i in range(30):
    print("**** Topic ",i," *****")
    print(lda.print_topic(i, topn=25))

In [ ]:
topic_distrib = [lda.get_document_topics(dictionary.doc2bow(commentaire)) for commentaire in sentences]

In [ ]:
len(topic_distrib)

In [ ]:
nbre_docs = len(topic_distrib)

doc_topic_matrix = np.zeros((nbre_docs,30))

for i,x in enumerate(topic_distrib):
    for j,p in x:
        doc_topic_matrix[i,j]=p

In [ ]:
len(doc_topic_matrix)

In [ ]:
from random import sample
from bhtsne import tsne
select_idx = sample(range(nbre_docs),114915)

tsne_proj = tsne(doc_topic_matrix[select_idx,:],perplexity=30.0)

In [ ]:
len(tsne_proj)

In [ ]:
proj_df = pd.DataFrame(data={"X_TSNE":tsne_proj[:,0], "Y_TSNE":tsne_proj[:,1],"COMMENT":data.iloc[select_idx]["COMMENT"].values})

In [ ]:
lda

In [ ]:
from sklearn.cluster import KMeans

N_clusters=35
kmeans_model = KMeans(n_clusters=N_clusters, random_state=np.random.randint(256),n_jobs=20,verbose=0).fit(lda)
classes = kmeans_model.labels_
proj_df["CLASS"] = classes

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
%matplotlib inline

colormap = get_cmap('magma')

plt.style.use('ggplot')

plt.figure(figsize=(5,5))
plt.scatter(proj_df["X_TSNE"],proj_df["Y_TSNE"],s=10,c=[colormap(x/30.0) for x in proj_df["CLASS"]])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
%matplotlib inline

for num_topic in range(35):
    print ("**** Topic ", num_topic,np.mean(doc_topic_matrix[:,num_topic]))
    colormap = get_cmap('inferno')

    plt.style.use('ggplot')

    plt.figure(figsize=(5,5))
    plt.scatter(tsne_proj[:,0],tsne_proj[:,1],s=10,c=[colormap(x/np.max(doc_topic_matrix[select_idx,num_topic])) for x in doc_topic_matrix[select_idx,num_topic]])
    plt.show()

In [ ]:
len(data)

In [ ]:
proj_df

In [ ]:
data["X_TSNE"]=proj_df["X_TSNE"]

In [ ]:
data["Y_TSNE"]=proj_df["Y_TSNE"]

In [ ]:
len(proj_df)

In [ ]:
### USE THIS CODE BELOW ONLY FOR VISUALIZING THE RESULTS IN THE JUPYTER NOTEBOOK

from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.layouts import vplot
import colorsys
from bokeh.models import HoverTool

output_notebook()

source = ColumnDataSource(
        data=dict(
            x=proj_df["X_TSNE"],
            y=proj_df["Y_TSNE"],
            commentaire=proj_df["COMMENT"]
        )
    )

hover = HoverTool(
        tooltips="""
                <p style="font-size: 10px;">@commentaire</p>
        """
    )


size_figs=800

p1 = figure(plot_width=size_figs, plot_height=size_figs, tools=["wheel_zoom,pan,lasso_select",hover])
p1.circle('x', 'y', source=source, size=5,line_color=None,fill_color="blue")

layout=hplot(p1)
show(layout)